Dependencies


In [1]:
!apt-get install -y ffmpeg
!pip install transformers torchaudio bitsandbytes


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
RECORDINGS_FOLDER = '/content/drive/MyDrive/Meet Recordings'


In [4]:
import os

entries = os.listdir(RECORDINGS_FOLDER)

print(entries[6])

adj-sdco-zab (2024-10-16 16:39 GMT-4)


In [21]:
video_filepath = f'/content/drive/MyDrive/Meet Recordings/{entries[6]}'
audio_filepath = '/content/drive/MyDrive/Meet Recordings/meeting_audio.mp3'


In [23]:
!ffmpeg -i "{video_filepath}" -q:a 0 -map a "{audio_filepath}" -y


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [24]:
import torch
from transformers import pipeline, AutoModelForSpeechSeq2Seq, AutoProcessor

In [25]:
AUDIO_MODEL = "openai/whisper-medium"
print("Loading ASR model...")
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    AUDIO_MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)
print("Model loaded!")





Loading ASR model...
Model loaded!


In [26]:
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device=0,
    return_timestamps=True
)

Device set to use cuda:0


In [27]:
def transcribe_audio(audio_path):
    print(f"Transcribing: {audio_path}")
    result = asr_pipeline(audio_path)
    transcript = result["text"]
    print("Transcription completed:")
    print(transcript)
    return transcript

transcript = transcribe_audio(audio_filepath)

Transcribing: /content/drive/MyDrive/Meet Recordings/meeting_audio.mp3


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Transcription completed:
 I will start recording, close that. Now you go to column, column. Now you go here, and you start your recording. OK. All right. So in three, two, one, go. Hello, everyone. I am Subankar Banerjee. I am doing a PhD in University of Maryland, College Park. This work I have done with my advisor, Professor Ulukush. So today I'm going to present my work, minimizing the age of information in an energy harvesting scheduler with redless codes. So in this work, we consider a source. The source simply governs a random process which this monitor wants to monitor. So for that, we have a sampler here. The job of the sampler is to sample a packet and transmit that packet to the monitor. So here, sampler and transmitter can be thought of as one block together. So sampler can act as transmitter as well. So sampler is equipped with, so we consider that this sampler is equipped with the energy harvesting battery. So we consider a slotted time in this world and we consider that e

In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


In [29]:
LLAMA_MODEL = "meta-llama/Meta-Llama-3.1-8B-Instruct"


In [30]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)


In [31]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LLAMA_MODEL,
    device_map="auto",
    quantization_config=quant_config
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [38]:
def generate_minutes(transcript_text):
    """
    Generate meeting minutes in markdown format from a transcript.
    The minutes include a summary, key discussion points, takeaways, and action items.
    """
    prompt = (
        "You are an assistant that produces meeting minutes from transcripts. "
        "Please generate meeting minutes in markdown format that strictly follow the template below:\n\n"
        "### Meeting Minutes\n\n"
        "**Summary:**\n"
        "- [Provide a concise summary of the meeting]\n\n"
        "**Discussion Points:**\n"
        "- [List each discussion point with details]\n\n"
        "**Takeaways:**\n"
        "- [List key takeaways from the meeting]\n\n"
        "**Action Items:**\n"
        "- [List action items along with the designated owners]\n\n"
        "Now, generate the meeting minutes for the following transcript:\n\n" + transcript_text +
        "\n\n### MEETING MINUTES START\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7
    )
    minutes = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return minutes

minutes_markdown = generate_minutes(transcript)
print("Generated Meeting Minutes:")
print(minutes_markdown)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generated Meeting Minutes:
You are an assistant that produces meeting minutes from transcripts. Please generate meeting minutes in markdown format that strictly follow the template below:

### Meeting Minutes

**Summary:**
- [Provide a concise summary of the meeting]

**Discussion Points:**
- [List each discussion point with details]

**Takeaways:**
- [List key takeaways from the meeting]

**Action Items:**
- [List action items along with the designated owners]

Now, generate the meeting minutes for the following transcript:

 I will start recording, close that. Now you go to column, column. Now you go here, and you start your recording. OK. All right. So in three, two, one, go. Hello, everyone. I am Subankar Banerjee. I am doing a PhD in University of Maryland, College Park. This work I have done with my advisor, Professor Ulukush. So today I'm going to present my work, minimizing the age of information in an energy harvesting scheduler with redless codes. So in this work, we consider

In [39]:
import smtplib
from email.mime.text import MIMEText


In [40]:
def send_email(subject, body, recipient, sender_email, sender_password):
    msg = MIMEText(body, "plain")
    msg["Subject"] = subject
    msg["From"] = sender_email
    msg["To"] = recipient

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, [recipient], msg.as_string())
    print(f"Email sent to {recipient}.")

separator = "### MEETING MINUTES START"
if separator in minutes_markdown:
    cleaned_minutes = minutes_markdown.split(separator, 1)[1].strip()
else:
    cleaned_minutes = minutes_markdown

SENDER_EMAIL = "sarangshibu98@gmail.com"
SENDER_PASSWORD = "**********************"
RECIPIENT_EMAIL = "sarangshibu98@gmail.com"

send_email("Meeting Minutes", cleaned_minutes, RECIPIENT_EMAIL, SENDER_EMAIL, SENDER_PASSWORD)





Email sent to sarangshibu98@gmail.com.
